In [1]:
import numpy as np
import pandas as pd
import os.path
import pickle
from scipy.stats import pearsonr, spearmanr

from cmapPy.pandasGEXpress.parse import parse # package from clue.io
import matplotlib.pyplot as plt

import numba as nb

In [2]:
@nb.njit(parallel=True)
def fastSort(a):
    b = np.empty(a.shape, dtype=np.int32)
    for i in nb.prange(a.shape[1]):
        b[:,i] = np.argsort(a[:,i])
    return b

def acc(P):
    """ accumulate matrix """
    for i in range(1, P.shape[0]):
        P[i] += P[i-1]
    return None
    
def ES(list_gene, profile):
    """ KS enrichment score"""
    if len(list_gene) == 0:
        return 0
    N, M = profile.shape[0], profile.shape[1]
    index = fastSort(-profile)
    rank = fastSort(index)
    # connectivity score in L1000 paper
    Nh = len(list_gene)
    Nr = np.zeros(M)
    P_hit = np.zeros((N, M))
    P_miss = np.ones((N, M))
    temp = np.array(range(M))
    profile = profile[index, temp]
    for i in list_gene:
        P_hit[rank[i], temp] = np.abs(profile[rank[i], temp])
        P_miss[rank[i], temp] = 0
        Nr += np.abs(profile[rank[i], temp])
    P_hit /= Nr
    P_miss /= N - Nh
    acc(P_hit)
    acc(P_miss)
    ES_score = P_hit - P_miss
    ES_score = ES_score[np.abs(ES_score).argmax(0), temp]

    return ES_score

def WTCS(list_up_index, list_dn_index, profile):
    """ weighted connectivity score """
    # shape: [978, M]
    batch_size = 10000
    M = profile.shape[1]
    num_batch = int(np.ceil(M / batch_size))
    wtcs_scores = np.zeros(M)
    for batch in range(num_batch):
        print("WTCS: {}/{}\r".format(batch+1, num_batch), end="")
        prof = profile[:, batch*batch_size:min(M, (batch+1)*batch_size)]
        ES_up = ES(list_up_index, prof)
        ES_dn = ES(list_dn_index, prof)
        wtcs = (ES_up - ES_dn) * 0.5 * (np.sign(ES_up) != np.sign(ES_dn))
        wtcs_scores[batch*batch_size:min(M, (batch+1)*batch_size)] = wtcs

    return wtcs_scores

In [3]:
####################################
# path to the LINCS data
####################################
path = '/data/tiantingzhong/LINCS2020/'

l1000file = "level5_beta_trt_cp_n720216x12328.gctx"
gene_info_file = "geneinfo_beta.txt"
sig_info_file  = "siginfo_beta_cp.txt"
pert_info_file = "compoundinfo_beta.txt"

In [4]:
pert_info = pd.read_csv(path + pert_info_file, sep="\t", low_memory=False)
gene_info = pd.read_csv(path + gene_info_file, sep="\t", dtype=str, low_memory=False)
sig_info = pd.read_csv(path + sig_info_file, sep="\t", low_memory=False)
sig_info = sig_info[sig_info['is_exemplar_sig'] == 1]
gene_landmark_rid = gene_info["gene_id"]
geneid2idx = {}
for i in gene_landmark_rid:
    geneid2idx[str(i)] = len(geneid2idx)

# Gene set of DC

In [5]:
up_geneset = [
    '7139', '3861', '800', '5210', '8061', '10205', '23452', '1783', '666', '7525', '4071', 
    '4313', '8425', '2919', '11167', '1848', '1291', '5654', '6662', '27289', '633', '3491', 
    '2615', '10626', '64787', '7077', '5742', '4131', '79971', '11127', '1893', '8848', '1514', 
    '51330', '2648', '7941', '79651', '9039', '1263', '1236', '1839', '1958', '6385', '822', 
    '5971', '664', '3949', '4792', '64114', '9123', '7378', '5211', '6513', '6275', '8870', 
    '894', '6277', '29923', '10982', '7167', '226', '836', '4794', '3659', '5315', '10938', 
    '4282', '4953', '4616', '9709'
] + ['23136', '5819', '4502']
dn_geneset = [
    '55723', '10403', '3835', '6402', '9133', '51172', '11133', '57405', '64924', '55143', 
    '23318', '84679', '374354', '332', '890', '891', '3009', '9212', '55872', '23623', '210', 
    '83461', '29071', '3575', '951', '10954', '3363', '3014', '23234', '10439', '3148', '221472', 
    '1152', '9555', '56650'
]
up_geneset = [geneid2idx[item] for item in up_geneset if item in geneid2idx]
dn_geneset = [geneid2idx[item] for item in dn_geneset if item in geneid2idx]
print('UP:', len(up_geneset))
print('DOWN:', len(dn_geneset))

UP: 73
DOWN: 35


In [6]:
new_index = up_geneset + dn_geneset 
old2new = {}
for i,j in enumerate(new_index):
    old2new[j] = i
gene_dc_rid = gene_landmark_rid[new_index]

In [7]:
L1000 = parse(path + l1000file, rid=gene_dc_rid)
L1000 = L1000.data_df.loc[gene_dc_rid][sig_info['sig_id']]
L1000 = np.array(L1000)
print(L1000.shape)

(108, 136460)


In [8]:
new_cs = WTCS([old2new[i] for i in up_geneset], 
               [old2new[i] for i in dn_geneset], 
               L1000)
new_rank = np.argsort(np.argsort(-new_cs)) + 1

In [9]:
sig_info_rank = sig_info.copy()
sig_info_rank['new_cs'] = new_cs
sig_info_rank['new_rank'] = new_rank
sig_info_rank = sig_info_rank.sort_values('new_cs', ascending=False)
list_core_col = ['cmap_name', 'pert_id', 'pert_idose', 'pert_itime', 'cell_iname', 'is_exemplar_sig', 
                        'new_cs', 'new_rank',]
sig_info_rank_core = sig_info_rank[list_core_col]
sig_info_rank_core = sig_info_rank_core[sig_info_rank_core['is_exemplar_sig']==1]
new_rank = np.argsort(np.argsort(-sig_info_rank_core['new_cs'])) + 1
sig_info_rank_core['new_rank'] = new_rank
pert_moa = pert_info[['pert_id', 'moa']].drop_duplicates()
pertid2moa = {}
for i in pert_moa.index:
    pertid = pert_moa.loc[i, 'pert_id']
    moa = pert_moa.loc[i, 'moa']
    if pertid not in pertid2moa:
        pertid2moa[pertid] = str(moa)
    else:
        pertid2moa[pertid] += ";"+str(moa)
list_pertid = list(pertid2moa.keys())
list_moa = list([pertid2moa[i] for i in list_pertid])
table_moa = pd.DataFrame(np.array([list_pertid, list_moa], dtype=object).T, columns=['pert_id', 'moa'])
sig_info_rank_core = pd.merge(sig_info_rank_core, table_moa, on='pert_id', how='left')

In [10]:
sig_info_rank_core[sig_info_rank_core['moa'] != 'nan'][:30]

,cmap_name,pert_id,pert_idose,pert_itime,cell_iname,is_exemplar_sig,new_cs,new_rank,moa
0,sitagliptin,BRD-K19416115,1.11 uM,24 h,MCF7,1,0.836546,1,Dipeptidyl peptidase inhibitor
2,GR-127935,BRD-K11911061,10 uM,24 h,MCF7,1,0.833276,3,Serotonin receptor antagonist
3,cyclosporin-a,BRD-A38030642,10 uM,6 h,A549,1,0.828587,4,Calcineurin inhibitor
16,VU-0418946-1,BRD-K44432556,10 uM,6 h,HEPG2,1,0.810447,17,HIF modulator
21,triciribine,BRD-A42649439,10 uM,24 h,MCF7,1,0.803583,22,Akt inhibitor
24,ingenol-mebutate,BRD-K93779381,0.125 uM,24 h,MDAMB231,1,0.800344,25,PKC activator
25,progesterone,BRD-K64994968,100 uM,24 h,MCF10A,1,0.798909,26,Progesterone receptor agonist
27,oxindole-I,BRD-K51816706,10 uM,24 h,MCF7,1,0.798562,28,VEGFR inhibitor
31,BRD-K00313977,BRD-K00313977,10 uM,24 h,MCF7,1,0.793562,32,APEX inhibitor
32,docetaxel,BRD-A05821830,0.04 uM,24 h,MCF10A,1,0.793056,33,Tubulin inhibitor


In [11]:
temp = sig_info_rank_core[sig_info_rank_core['moa'] != 'nan']
temp_show = temp[:30][['cmap_name', 'pert_id', 'new_cs', 'moa']]
temp_show.columns = ['Compound name', 'Compound ID', 'Screening score', 'MOA']
temp_show = temp_show.reset_index(drop=True)
temp_show

,Compound name,Compound ID,Screening score,MOA
0,sitagliptin,BRD-K19416115,0.836546,Dipeptidyl peptidase inhibitor
1,GR-127935,BRD-K11911061,0.833276,Serotonin receptor antagonist
2,cyclosporin-a,BRD-A38030642,0.828587,Calcineurin inhibitor
3,VU-0418946-1,BRD-K44432556,0.810447,HIF modulator
4,triciribine,BRD-A42649439,0.803583,Akt inhibitor
5,ingenol-mebutate,BRD-K93779381,0.800344,PKC activator
6,progesterone,BRD-K64994968,0.798909,Progesterone receptor agonist
7,oxindole-I,BRD-K51816706,0.798562,VEGFR inhibitor
8,BRD-K00313977,BRD-K00313977,0.793562,APEX inhibitor
9,docetaxel,BRD-A05821830,0.793056,Tubulin inhibitor
